<a href="https://colab.research.google.com/github/safae-rabbouzi/Text-preprocessing/blob/main/Text_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download -d thoughtvector/customer-support-on-twitter

 94% 159M/169M [00:01<00:00, 106MB/s]
100% 169M/169M [00:01<00:00, 111MB/s]


In [9]:
!unzip /content/customer-support-on-twitter.zip

Archive:  /content/customer-support-on-twitter.zip
  inflating: sample.csv              
  inflating: twcs/twcs.csv           


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
import string
import spacy
import re
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
df=pd.read_csv('/content/sample.csv')

In [13]:
df

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0
...,...,...,...,...,...,...,...
88,119330,105859,True,Wed Oct 11 13:50:42 +0000 2017,@105860 I wish Amazon had an option of where I...,119329,119331.0
89,119331,105860,True,Wed Oct 11 13:47:14 +0000 2017,They reschedule my shit for tomorrow https://t...,119330,NaN
90,119332,Tesco,False,Wed Oct 11 13:34:06 +0000 2017,"@105861 Hey Sara, sorry to hear of the issues ...",119333,119334.0
91,119333,105861,True,Wed Oct 11 14:05:18 +0000 2017,@Tesco bit of both - finding the layout cumber...,"119335,119336",119332.0


In [14]:
df["text"]

0     @AppleSupport causing the reply to be disregar...
1     @105835 Your business means a lot to us. Pleas...
2     @76328 I really hope you all change but I'm su...
3     @105836 LiveChat is online at the moment - htt...
4     @VirginTrains see attached error message. I've...
                            ...                        
88    @105860 I wish Amazon had an option of where I...
89    They reschedule my shit for tomorrow https://t...
90    @105861 Hey Sara, sorry to hear of the issues ...
91    @Tesco bit of both - finding the layout cumber...
92    @105861 If that doesn't help please DM your fu...
Name: text, Length: 93, dtype: object

In [15]:
df['text_lower'] = df['text'].str.lower()
print(df['text_lower'])

0     @applesupport causing the reply to be disregar...
1     @105835 your business means a lot to us. pleas...
2     @76328 i really hope you all change but i'm su...
3     @105836 livechat is online at the moment - htt...
4     @virgintrains see attached error message. i've...
                            ...                        
88    @105860 i wish amazon had an option of where i...
89    they reschedule my shit for tomorrow https://t...
90    @105861 hey sara, sorry to hear of the issues ...
91    @tesco bit of both - finding the layout cumber...
92    @105861 if that doesn't help please dm your fu...
Name: text_lower, Length: 93, dtype: object


In [16]:
translator = str.maketrans('', '', string.punctuation)
df['text_punc'] = df['text_lower'].apply(lambda x: x.translate(translator))
print(df['text_punc'])

0     applesupport causing the reply to be disregard...
1     105835 your business means a lot to us please ...
2     76328 i really hope you all change but im sure...
3     105836 livechat is online at the moment  https...
4     virgintrains see attached error message ive tr...
                            ...                        
88    105860 i wish amazon had an option of where i ...
89    they reschedule my shit for tomorrow httpstcor...
90    105861 hey sara sorry to hear of the issues yo...
91    tesco bit of both  finding the layout cumberso...
92    105861 if that doesnt help please dm your full...
Name: text_punc, Length: 93, dtype: object


In [17]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["text_stopWors"] = df["text_punc"].apply(lambda text: remove_stopwords(text))
print(df["text_stopWors"])

0     applesupport causing reply disregarded tapped ...
1     105835 business means lot us please dm name zi...
2            76328 really hope change im sure wont dont
3     105836 livechat online moment httpstcosy94vtu8...
4     virgintrains see attached error message ive tr...
                            ...                        
88    105860 wish amazon option get shipped ups stor...
89          reschedule shit tomorrow httpstcorsvzct982t
90    105861 hey sara sorry hear issues ask lay spee...
91    tesco bit finding layout cumbersome removing i...
92    105861 doesnt help please dm full name address...
Name: text_stopWors, Length: 93, dtype: object


In [18]:
from collections import Counter
all_text = ' '.join(df['text'])
words = all_text.split()
word_freq = Counter(words)

common_words = word_freq.most_common(10)

stop_words = [word for word, freq in common_words]

def remove_common_words(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

df['text_commonWords'] = df['text_stopWors'].apply(remove_common_words)

print(df['text_commonWords'])

0     applesupport causing reply disregarded tapped ...
1     105835 business means lot us please dm name zi...
2            76328 really hope change im sure wont dont
3     105836 livechat online moment httpstcosy94vtu8...
4     virgintrains see attached error message ive tr...
                            ...                        
88    105860 wish amazon option get shipped ups stor...
89          reschedule shit tomorrow httpstcorsvzct982t
90    105861 hey sara sorry hear issues ask lay spee...
91    tesco bit finding layout cumbersome removing i...
92    105861 doesnt help please dm full name address...
Name: text_commonWords, Length: 93, dtype: object


In [19]:
stop_words = set(stopwords.words('english'))

tokenized_words = [word.lower() for text in df['text'] for word in word_tokenize(text) if word.isalpha() and word.lower() not in stop_words]

word_freq = FreqDist(tokenized_words)

min_freq = 2

filtered_words = [word for word in word_freq if word_freq[word] >= min_freq]

def remove_rare_words(text, rare_words):
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() in rare_words]
    return ' '.join(filtered_words)
df['text_rareWords'] = df['text_commonWords'].apply(remove_rare_words, rare_words=filtered_words)

print(df['text_rareWords'])

0                                    applesupport reply
1              means lot us please dm name code concern
2                                             hope sure
3                       moment option message call back
4     virgintrains see message tried several times p...
                            ...                        
88                             option get ups store lot
89                                                     
90                        hey sorry hear issues website
91                                        tesco getting
92    help please dm full name address email device ...
Name: text_rareWords, Length: 93, dtype: object


In [20]:
lemmatizer = WordNetLemmatizer()

def apply_lemmatization(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

df['text_lemm'] = df['text_rareWords'].apply(apply_lemmatization)

print(df['text_lemm'])

0                                    applesupport reply
1                mean lot u please dm name code concern
2                                             hope sure
3                       moment option message call back
4     virgintrains see message tried several time pa...
                            ...                        
88                             option get ups store lot
89                                                     
90                         hey sorry hear issue website
91                                        tesco getting
92    help please dm full name address email device ...
Name: text_lemm, Length: 93, dtype: object


In [21]:
stemmer = SnowballStemmer('english')

def apply_stemming(text):
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

df['text_stemmed'] = df['text_lemm'].apply(apply_stemming)
print(df['text_stemmed'])

0                                    applesupport repli
1                 mean lot u pleas dm name code concern
2                                             hope sure
3                        moment option messag call back
4       virgintrain see messag tri sever time past week
                            ...                        
88                              option get up store lot
89                                                     
90                           hey sorri hear issu websit
91                                            tesco get
92    help pleas dm full name address email devic thank
Name: text_stemmed, Length: 93, dtype: object


In [22]:
import re

def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # Emojis faces
                               u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # Transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # Flags (iOS)
                               u"\U00002500-\U00002BEF"  # Chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_html_tags(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

df['text_remEmoj'] = df['text_stemmed'].apply(remove_emojis)
df['text_remUrl'] = df['text_remEmoj'].apply(remove_urls)
df['text_remHtml'] = df['text_remUrl'].apply(remove_html_tags)

print(df['text_remHtml'])

0                                    applesupport repli
1                 mean lot u pleas dm name code concern
2                                             hope sure
3                        moment option messag call back
4       virgintrain see messag tri sever time past week
                            ...                        
88                              option get up store lot
89                                                     
90                           hey sorri hear issu websit
91                                            tesco get
92    help pleas dm full name address email devic thank
Name: text_remHtml, Length: 93, dtype: object


In [25]:
df['text_cleaned'] = df['text_remHtml']

In [26]:
df_result = df[['tweet_id', 'text', 'text_cleaned']]

In [27]:
df_result

,tweet_id,text,text_cleaned
0,119237,@AppleSupport causing the reply to be disregar...,applesupport repli
1,119238,@105835 Your business means a lot to us. Pleas...,mean lot u pleas dm name code concern
2,119239,@76328 I really hope you all change but I'm su...,hope sure
3,119240,@105836 LiveChat is online at the moment - htt...,moment option messag call back
4,119241,@VirginTrains see attached error message. I've...,virgintrain see messag tri sever time past week
...,...,...,...
88,119330,@105860 I wish Amazon had an option of where I...,option get up store lot
89,119331,They reschedule my shit for tomorrow https://t...,
90,119332,"@105861 Hey Sara, sorry to hear of the issues ...",hey sorri hear issu websit
91,119333,@Tesco bit of both - finding the layout cumber...,tesco get
